In [36]:
%load_ext autoreload
%autoreload 2

import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
import plotly.subplots as sp

df = pd.DataFrame
sns.set_theme()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
w,x,wxb = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 7,
    outBits = 4,
    seed = 0,
    weight_mode = 'binary',
    col_symmetric=False,
    clip_output = False
)

rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtl_binary.txt')
ref = wxb.T[::-1].T

snr = np.var(ref) / np.var(ref - rtl)
snr = 10 * np.log10(snr)

fig = px.scatter(x=rtl.flatten(), y=ref.flatten(), title=f'SNR:{snr:.2f} dB', labels={'x':'RTL', 'y':'Python'},
    width=400,height=400
)
# Add diagonal line
limit = 1000
fig.add_shape(
    type="line", line=dict(dash="dash"),
    x0=-limit, y0=-limit, x1=limit, y1=limit
)


[STIM_GEN] Generating random weights


In [47]:
w,x,wxb = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 7,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False,
    clip_output = False
)

rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtl_bipolar.txt')
ref = wxb.T[::-1].T

snr = np.var(ref) / np.var(ref - rtl)
snr = 10 * np.log10(snr)

fig = px.scatter(x=rtl.flatten(), y=ref.flatten(), title=f'SNR:{snr:.2f} dB', labels={'x':'RTL', 'y':'Python'},
    width=400,height=400
)
# Add diagonal line
limit = 1000
fig.add_shape(
    type="line", line=dict(dash="dash"),
    x0=-limit, y0=-limit, x1=limit, y1=limit
)


[STIM_GEN] Generating random weights
